In [2]:
import json
import random
from latools import *
from sympy import *
init_printing(use_latex=True)

In [2]:
def rref_to_solution_basis(R):
    m, n = R.shape
    nvars = n - 1
    jold = -1
    pivots = []
    zero_rows = []
    for i in range(m):
        for j in range(n-1):
            if R[i,j] != 0:
                break
        else:
            zero_rows.append(i)
            continue
        if len(zero_rows) != 0:
            raise ValueError('zero rows not at the bottom')
        if R[i,j] != 1:
            raise ValueError('leading element of row {} is not 1'.format(i+1))
        for k in range(m):
            if k != i and R[k, j] != 0:
                raise ValueError('nonzero in pivot column {}'.format(j+1))
        if j <= jold:
            raise ValueError('pivot columns not in increasing order')
        pivots.append((i,j))
    for i in zero_rows:
        if R[i,nvars] != 0:
            return None, None
    pivot_columns = [j for _ , j in pivots]
    non_pivot_columns = [j for j in range(n-1) if j not in pivot_columns]
    cvec = zeros(nvars, 1)
    for (i,j) in pivots:
        cvec[j] = R[i, n-1]
    bvecs = [zeros(nvars,1) for _ in range(len(non_pivot_columns))]
    for i, j in enumerate(non_pivot_columns):
        bvecs[i][j] = 1
        for k, l in pivots:
            bvecs[i][l] = -R[k,j]
    return cvec, bvecs
def vector_to_latex(vec, align=''):
    if len(align) > 0:
        start = '\\begin{{bmatrix*}}[{}]'.format(align)
        end =  '\\end{bmatrix*}'
    else:
        start = '\\begin{bmatrix}'
        end = '\\end{bmatrix}'
    return start + '\\\\'.join(str(v) for v in vec) + end
def rref_to_solution_latex(R, align=''):
    cvec, bvecs = rref_to_solution_basis(R)
    if cvec is None:
        return '\\varnothing'
    sout = '\\left\\{' + vector_to_latex(cvec, align=align) + '\n' 
    for i, vec in enumerate(bvecs):
        sout += '+\\lambda_{{{}}}'.format(i+1) + vector_to_latex(vec, align=align) + '\n'
    sout += '\\;:\\;'
    sout += ','.join('\\lambda_{{{}}}'.format(i+1) for i in range(len(bvecs)))
    sout += '\\in\\mathbb{R}\\right\\}\n'
    return sout
def make_linear_system_exercise(R, pvalues=range(-4,4), align='', vnames=None):
    m, n = R.shape
    P = zeros(m)
    while P.det() == 0:
        for i in range(m):
            for j in range(m):
                P[i, j] = nsimplify(random.choice(pvalues))
    A = P * R
    system_str = matrix_to_system_latex(A, vnames)
    solution_str = rref_to_solution_latex(R, align=align)
    return system_str, solution_str
def pattern_to_exercise(pattern, rvalues=range(-5,5), pvalues=range(-3,3), align=''):
    m = len(pattern)
    n = max(len(pat) for pat in pattern)
    R = zeros(m, n)
    for i, pat in enumerate(pattern):
        for j, s in enumerate(pat):
            R[i,j] = 0 if s == '0' else 1 if s == '1' else random.choice(rvalues)
    return make_linear_system_exercise(R, pvalues, align=align)

Tests

In [3]:
pattern = ["1x0x00xx",
      "001x00xx",
      "000010xx",
      "000001xx"
     ]

In [4]:
exercise, solution = pattern_to_exercise(pattern)
print(exercise)
print(solution)

\begin{alignat*}{17}
&{}{}&&{}{}&- 3 x_{3} &{}-{}& 12 x_{4} &{}-{}& 2 x_{5}&{}{}& &{}-{}& 12 x_{7} &{}={}&4\\ 
x_{1}&{}{}& &{}-{}& 3 x_{3} &{}-{}& 11 x_{4} &{}+{}& 2 x_{5} &{}-{}& 2 x_{6} &{}-{}& 4 x_{7} &{}={}&-1\\ 
- 2 x_{1}&{}{}& &{}-{}& 3 x_{3} &{}-{}& 14 x_{4} &{}+{}& x_{5}&{}{}& &{}-{}& 11 x_{7} &{}={}&17\\ 
- 2 x_{1}&{}{}& &{}-{}& 3 x_{3} &{}-{}& 14 x_{4} &{}-{}& x_{5} &{}-{}& x_{6} &{}-{}& 21 x_{7} &{}={}&13\\ 
\end{alignat*}

\left\{\begin{bmatrix}-5\\0\\-2\\0\\1\\2\\0\end{bmatrix}
+\lambda_{1}\begin{bmatrix}0\\1\\0\\0\\0\\0\\0\end{bmatrix}
+\lambda_{2}\begin{bmatrix}-1\\0\\-4\\1\\0\\0\\0\end{bmatrix}
+\lambda_{3}\begin{bmatrix}-4\\0\\-2\\0\\-3\\-4\\1\end{bmatrix}
\;:\;\lambda_{1},\lambda_{2},\lambda_{3}\in\mathbb{R}\right\}



This is a system:
\begin{alignat*}{17}
&{}{}&&{}{}&- 3 x_{3} &{}+{}& 6 x_{4} &{}-{}& 2 x_{5} &{}+{}& 2 x_{6} &{}-{}& 9 x_{7} &{}={}&-12\\ 
- x_{1} &{}-{}& x_{2}&{}{}& &{}-{}& 4 x_{4} &{}-{}& 3 x_{5} &{}+{}& 2 x_{6} &{}-{}& 10 x_{7} &{}={}&-19\\ 
- 2 x_{1} &{}-{}& 2 x_{2} &{}-{}& x_{3} &{}-{}& 6 x_{4} &{}+{}& x_{5} &{}-{}& x_{6} &{}+{}& 15 x_{7} &{}={}&0\\ 
x_{1} &{}+{}& x_{2} &{}-{}& 2 x_{3} &{}+{}& 8 x_{4}&{}{}& &{}+{}& x_{6} &{}-{}& 6 x_{7} &{}={}&1\\ 
\end{alignat*}


$$
\left\{\begin{bmatrix}1\\0\\-5\\0\\-2\\-5\\0\end{bmatrix}
+\lambda_{1}\begin{bmatrix}-2\\1\\0\\0\\0\\0\\0\end{bmatrix}
+\lambda_{2}\begin{bmatrix}-2\\0\\0\\1\\0\\0\\0\end{bmatrix}
+\lambda_{3}\begin{bmatrix}3\\0\\-2\\0\\5\\5\\1\end{bmatrix}
\;:\;\lambda_{1},\lambda_{2},\lambda_{3}\in\mathbb{R}\right\}
$$

In [7]:
dirname = 'make-linear-system'
in_file = open(dirname + '/rref_patterns.json','r')
patterns = json.load(in_file)
problems = []
solutions = []
for pattern in patterns:
    print(pattern)
    problem, solution = pattern_to_exercise(pattern, align='r')
    problems.append(problem)
    solutions.append(solution)
in_file.close()
out_str = """
\\documentclass[12pt]{article}
\\input{../lamacros.tex}
\\setheadings{MTH288 --- Exercises --- Solving Linear Systems}
\\begin{document}\n
For each of the items below, do the following:
\\begin{enumerate}
\\item Write the augmented matrix for the given system.
\\item Use elementary row operations to transform the matrix into reduced row echelon form.
\\item Write the system that corresponds to the reduced row echelon form.
\\item Write the solution set of the system (using set notation).
\\end{enumerate}

\\textbf{Exercises}:
\\begin{enumerate}
"""
for problem in problems:
    out_str += '\\item ' + problem + '\n'
out_str += """
\\end{enumerate}
\\clearpage
\\textbf{Solutions}:
\\begin{enumerate}
"""
for solution in solutions:
    out_str += '\\item\\[' + solution + '\\]\n'
out_str += """
\\end{enumerate}
\\end{document}
"""
out_file = open(dirname + '/linear_system_exercises.tex', 'w')
out_file.write(out_str)
out_file.close()

['1x0x0xx', '001x0xx', '00001xx', '0000000']
['1000x0x', '0100x0x', '0010x0x', '0001x0x', '000001x']
['1000x', '0100x', '0010x', '0001x', '00000']
